<a href="https://colab.research.google.com/github/NhungNguyen04/Clothing-shop-be/blob/master/StableVITON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rlawjdghek/StableVITON.git
%cd StableVITON

Cloning into 'StableVITON'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 166 (delta 29), reused 123 (delta 13), pack-reused 1 (from 1)
Receiving objects: 100% (166/166), 2.44 MiB | 7.97 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/StableVITON


In [ ]:
!pip uninstall -y torch torchvision torchaudio torchtext
!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 torchtext==0.14.0
!pip install pytorch-lightning==1.5.0 einops opencv-python matplotlib omegaconf albumentations transformers==4.33.2
!pip install xformers==0.0.19 triton==2.0.0 open-clip-torch==2.19.0 diffusers==0.20.2 scipy==1.10.1

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.15.0
ERROR: Could not find a version that satisfies the requirement torchvision==0.14.1 (from versions: 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.16.2, 0.17.0, 0.17.1, 0.17.2, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 0.20.0, 0.20.1, 0.21.0)
ERROR: No matching distribution found for torchvision==0.14.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.1 MB/s eta 0:00:00
  Preparing metada

In [ ]:
# Create the checkpoints directory
!mkdir -p ckpts

# Download the pre-trained model weights
!wget -O ckpts/VITONHD_PBE_pose.ckpt https://huggingface.co/rlawjdghek/StableVITON/blob/main/ckpts/VITONHD_PBE_pose.ckpt
!wget -O ckpts/VITONHD_VAE_finetuning.ckpt https://huggingface.co/rlawjdghek/StableVITON/blob/main/ckpts/VITONHD_VAE_finetuning.ckpt

In [ ]:
!mkdir -p test/image
!mkdir -p test/cloth
!mkdir -p test/image-densepose
!mkdir -p test/agnostic
!mkdir -p test/agnostic-mask
!mkdir -p test/cloth_mask

In [ ]:
from google.colab import files


print("Upload test images")
uploaded_images = files.upload()
for filename in uploaded_images.keys():
    !mv {filename} test/image/

print("Upload clothing images")
uploaded_cloths = files.upload()
for filename in uploaded_cloths.keys():
    !mv {filename} test/cloth/

In [ ]:
import os

image_files = os.listdir('test/image')
cloth_files = os.listdir('test/cloth')

print(f"Number of test images: {len(image_files)}")
print(f"Number of cloth images: {len(cloth_files)}")

if len(image_files) != len(cloth_files):
    print("Error: The number of image files and cloth files must be the same.")
else:
    with open('test/test_pairs.txt', 'w') as f:
        for image_file, cloth_file in zip(image_files, cloth_files):
            f.write(f"{image_file} {cloth_file}\n")

In [ ]:
!python /content/StableVITON/inference.py --config_path ./StableVITON/configs/VITONHD.yaml --batch_size 4 --model_load_path ./ckpts/VITONHD_PBE_pose.ckpt --save_dir result